In [10]:
from __future__ import print_function, division

# Standard imports to work with datasets, plots, etc.
import pylab as pl
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import json
import requests
import urllib2
from pandas.tools.plotting import scatter_matrix
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.interpolate import *

#loading rcParams from JSON template (created from Federica's fbb_matplotlibrc.json)
url = 'https://s3.amazonaws.com/sb-public/sbg389_matplotlibrc.json'
resp = requests.get(url=url)
data = json.loads(resp.text)
# update the rcParams object
pl.rcParams.update(data)

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
# Getting and move things around with bash commands
# Download the manhattan pulto shape file
# Created a MNMapPluto folder under PUIData to put all the Pluto Files

# Use os.system to invoke bash commands to download (curl) unzip and move file

# Curl -O Write output to a local file named like the remote file we get
os.system("curl -O https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/mn_mappluto_16v1.zip")

# Unzip the files and move to the MNMapPluto subfolder (they all start with MN)
os.system("unzip -jn mn_mappluto_16v1.zip MN*")

#Move files to our PUIDATA
os.system("mv " + "MN* " + os.getenv("PUIDATA") + "/MNMapPluto")

# Read the Manhattan SHP file that was obtained from the PLUTO dataset
# geopandas (or pandas, its the same) reading from local file
bsize = gpd.read_file (os.getenv("PUIDATA") + "/MNMapPluto/MNMapPLUTO.shp")
bsize.head(2)


In [22]:
# Read a csv directly from the web (both geopandas and pandas are ok with it)
nrg = pd.read_csv("https://data.cityofnewyork.us/api/views/rgfe-8y2z/rows.csv")

In [14]:
# Obtaining Data from a JSON API, putting it into a dictionary and reading a specific value

MTAKEY = 'e106ffee-0fd1-4f89-bcda-30cf0d9e50c6'
BUSLINE = 'B62'

#Build the URI for the API Call concatenating the key and bus line from the argument parameters
url = "http://bustime.mta.info/api/siri/vehicle-monitoring.json?key=%s&VehicleMonitoringDetailLevel=calls&" \
      "LineRef=%s"%(MTAKEY, BUSLINE)

#Get the response and load the string representation into a dictionary
response = urllib2.urlopen(url)
mtadataString = response.read().decode("utf-8")
mtadata = json.loads(mtadataString)

vehicleActivityArray = mtadata['Siri']['ServiceDelivery']['VehicleMonitoringDelivery']
numberOfActiveBuses = len(vehicleActivityArray[0]['VehicleActivity'])

# print (vehicleActivityArray)
print (numberOfActiveBuses)

7


In [30]:
# Clean up the dataframe:

# List columns (so we can drop etc.)

# List returns a list with the column names, that can be used to drop, select, etc.
print (list (nrg.columns))

# Rename Columns (so they can be called using dots and avoid errros with non ascii chars on the names (if any))
#change Borough Block and Lot to BBL to work as a merge key 
nrg.rename(columns={'nyc_borough_block_and_lot_bbl': 'BBL'}, inplace=True)

print ('')
print (list (nrg.columns))

['Record Number', 'NYC Borough, Block, and Lot (BBL)', 'Co-reported BBL Status', 'BBLs Co-reported', 'Reported NYC Building Identificaiton Numbers (BINs)', 'Street Number', 'Street Name', 'Borough', 'Zip Code', 'DOF Benchmarking Submission Status', 'Site EUI(kBtu/ft2)', 'Weather Normalized Site EUI(kBtu/ft2)', 'Source EUI(kBtu/ft2)', 'Weather Normalized Source EUI(kBtu/ft2)', 'Municipally Supplied Potable Water - Indoor Intensity (gal/ft\xc2\xb2)', 'Automatic Water Benchmarking Eligible', 'Reported Water Method', 'ENERGY STAR Score', 'Total GHG Emissions(MtCO2e)', 'Direct GHG Emissions(MtCO2e)', 'Indirect GHG Emissions(MtCO2e)', 'Reported Property Floor Area (Building(s)) (ft\xc2\xb2)', 'DOF Property Floor Area (Buildngs and Parking)(ft2)', 'Primary Property Type - Self Selected', 'DOF Number of Buildings']

['Record Number', 'NYC Borough, Block, and Lot (BBL)', 'Co-reported BBL Status', 'BBLs Co-reported', 'Reported NYC Building Identificaiton Numbers (BINs)', 'Street Number', 'Street